In [6]:
import sys, string
import os
import socket
import time
from time import gmtime, strftime
import operator
import boto3
import json
import findspark
from pyspark.sql import SparkSession
from datetime import datetime


In [7]:
findspark.init()

In [8]:
spark = SparkSession\
        .builder\
        .appName("ethereum-sneha-D")\
        .getOrCreate()

In [9]:
def filter_transaction_line(line):
    try:
        fields = line.split(',')
        if len(fields) != 7 or fields[0] == 'block_number':
            return False
        else:
            str(fields[2]) # convert to_address to string 
            int(fields[5]) # convert gas price to float
            int(fields[6]) # convert timestamp to int
            return True
    except:
        return False
        


    


In [12]:
transactions_file = spark.sparkContext.textFile("./transactions.csv").filter(filter_transaction_line)
# calculate value received at each address in each month
weekly_gas_price_map = transactions_file.map(lambda x: (strftime('%Y-%m W%W', gmtime(int(x.split(',')[6]))), (int(x.split(',')[5]), 1)))

In [14]:
weekly_gas_price_map.collect()[:5]

[('2016-09 W37', (20000000000, 1)),
 ('2016-09 W37', (20000000000, 1)),
 ('2016-09 W37', (20000000000, 1)),
 ('2016-09 W37', (20000000000, 1)),
 ('2016-09 W37', (20000000000, 1))]

In [23]:
weekly_gas_price_reduce = weekly_gas_price_map.reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1]))
# weekly_gas_price_reduce = weekly_gas_price_map.reduceByKey(lambda a, b: a+b)

remap = weekly_gas_price_reduce.mapValues(lambda x: int(round(x[0]/x[1]/1000000000)) if x[1] > 0 else 0)
# remap = weekly_gas_price_reduce.mapValues(lambda x: int(round(x[0]/x[1])) if x[1] > 0 else 0)

sorted_weekly_gas_price = remap.sortByKey()

In [24]:
sorted_weekly_gas_price.collect()

[('2016-09 W37', 23)]